## 0. Импорты

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
from catboost import CatBoostRegressor, Pool
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

## 1. Чтение данных и визуализация

In [2]:
df_train = pd.read_parquet('train.parquet', engine='fastparquet')
df_test = pd.read_parquet('test.parquet', engine='fastparquet')
df_sample = pd.read_csv('sample_submission.csv', index_col=False, parse_dates=['dt'])

print(df_train.shape)
print(df_test.shape)
print(df_train['dt'].min())
print(df_test['dt'].max())

(309648, 6)
(12856, 5)
2024-07-04 00:00:00
2025-07-21 00:00:00


In [3]:
print(f'Количество уникальных товаров в ТЕСТ: {df_test["nm_id"].nunique()}')
print(f'Количество уникальных дней в ТЕСТ: {df_test["dt"].nunique()}')

print(f'Количество уникальных товаров в ТРЕЙН: {df_train["nm_id"].nunique()}')
print(f'Количество уникальных дней в ТРЕЙН: {df_train["dt"].nunique()}')

id_train = df_train['nm_id'].unique().tolist()
id_test = df_test['nm_id'].unique().tolist()

only_test_ids = list(set(id_test) - set(id_train))
print(f'товары которых попали только в ТЕСТ {only_test_ids}')

Количество уникальных товаров в ТЕСТ: 949
Количество уникальных дней в ТЕСТ: 14
Количество уникальных товаров в ТРЕЙН: 2743
Количество уникальных дней в ТРЕЙН: 369
товары которых попали только в ТЕСТ ['9ccb2ba0-8584-4104-879b-c8d26db6ea8a', 'a1cad3b9-f34b-4b23-8e74-b927b0730982']


In [4]:
# Визуализация остатков и количество купленных товаров на всей выборки по конкретному товару
def visualiser(nm_id):
    df = pd.concat([df_train, df_test], ignore_index=True)
    item_data = df[df['nm_id'] == nm_id].sort_values('dt')

    train_max_date = df['dt'].max() - pd.Timedelta(days=14)
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Train бары
    fig.add_trace(go.Bar(x=item_data['dt'], y=item_data['qty'], 
                        name='Продажи train', marker_color='green', opacity=0.5),
                 secondary_y=False)
    
    # ЛИНИЯ РАЗДЕЛИТЕЛЯ (add_shape вместо add_vline)
    fig.add_shape(type="line",
                  x0=train_max_date, x1=train_max_date,
                  y0=0, y1=1, yref="paper",
                  line=dict(color="red", width=3, dash="dash"))

    
    # Остатки
    fig.add_trace(go.Bar(x=item_data['dt'], y=item_data['prev_leftovers'].fillna(0),
                        name='Остатки train+test', marker_color='red', opacity=0.5),
                 secondary_y=True)
    
    # Настройка
    fig.update_layout(
        title=f'Продажи и остатки товара {nm_id}',
        hovermode='x unified',
        legend=dict(x=0.01, y=0.99),
        height=500,
        showlegend=True
    )
    fig.update_xaxes(title_text="Дата")
    fig.update_yaxes(title_text="Продажи (qty)", secondary_y=False)
    fig.update_yaxes(title_text="Остатки (prev_leftovers)", secondary_y=True)
    
    fig.show()


In [5]:
nm_id = df_test['nm_id'].unique()[11] # 9
visualiser(nm_id)

In [6]:
nm_id = '5dcc404b-aea3-42d9-8b1c-a2379e5e19c7'
visualiser(nm_id)

## 2. Генерация признаков

In [7]:
# Для начала объединим тренировочные и тестовые данные для feature enginiring
df = pd.concat([df_train, df_test], ignore_index=True)
df

,nm_id,dt,qty,price,is_promo,prev_leftovers
0,015ecac1-0d4d-4272-9f9c-7c62a742a679,2024-07-04,0.0,2940,0,54
1,02790618-93a6-4a2e-92e2-72cbb0bf24cf,2024-07-04,3.0,6543,0,75
2,03cf7d0c-4709-409d-a964-44558ada7e3d,2024-07-04,3.0,4983,0,66
3,0407f293-33e5-4c34-8236-9ddef5a166f0,2024-07-04,3.0,1971,1,132
4,046336e9-58cd-4d0f-b521-c70607fd9ec4,2024-07-04,0.0,2133,1,87
...,...,...,...,...,...,...
322499,fe44ffe3-9dcd-4f36-b212-a0f7f35776d9,2025-07-21,NaN,8727,1,12
322500,fef549dd-9fe5-4cf2-b9f0-2539c103a4f2,2025-07-21,NaN,10362,1,36
322501,ff0d747b-bdc1-4628-8163-8fcc27eed4bc,2025-07-21,NaN,8727,0,84
322502,ff518e5b-e28f-41f9-9610-bf9f6bdbe334,2025-07-21,NaN,40302,1,102


In [8]:
# Как показали эксперименты в данной задаче лучше оставить только те товары которые есть в тетсовой выборке
df = df[df['nm_id'].isin(df_test['nm_id'].unique().tolist())].reset_index(drop=True)
df

,nm_id,dt,qty,price,is_promo,prev_leftovers
0,068272a6-6a24-4918-b7ad-b24b1fe9d8ee,2024-07-04,0.0,27192,0,12
1,08154c00-83b8-470b-85ff-cadddd2eca71,2024-07-04,0.0,13356,0,141
2,13b87038-e487-4430-aa5a-d0ad14c7eb5f,2024-07-04,0.0,4878,1,693
3,154d1322-bd30-496c-9374-6dbd35550669,2024-07-04,0.0,3966,1,1314
4,182704f4-fc4b-46f1-be35-1934d04be1b9,2024-07-04,0.0,9795,0,621
...,...,...,...,...,...,...
163098,fe44ffe3-9dcd-4f36-b212-a0f7f35776d9,2025-07-21,NaN,8727,1,12
163099,fef549dd-9fe5-4cf2-b9f0-2539c103a4f2,2025-07-21,NaN,10362,1,36
163100,ff0d747b-bdc1-4628-8163-8fcc27eed4bc,2025-07-21,NaN,8727,0,84
163101,ff518e5b-e28f-41f9-9610-bf9f6bdbe334,2025-07-21,NaN,40302,1,102


In [9]:
# генерируем первую ппачку признаков

df = df.sort_values(['nm_id', 'dt']).reset_index(drop=True)


df['prev_leftovers_future_2'] = df.groupby('nm_id')['prev_leftovers'].shift(-2)
df['prev_leftovers_future_1'] = df.groupby('nm_id')['prev_leftovers'].shift(-1)

df['temp_2'] = df['prev_leftovers_future_1'] - df['prev_leftovers_future_2']
df['temp_2'] = df['temp_2'].fillna(0)

df['temp_1'] = df['prev_leftovers'] - df['prev_leftovers_future_1']
df['temp_1'] = df['temp_2'].fillna(0)



df['prev_leftovers_past_4'] = df.groupby('nm_id')['prev_leftovers'].shift(4)
df['prev_leftovers_past_3'] = df.groupby('nm_id')['prev_leftovers'].shift(3)
df['prev_leftovers_past_2'] = df.groupby('nm_id')['prev_leftovers'].shift(2)
df['prev_leftovers_past_1'] = df.groupby('nm_id')['prev_leftovers'].shift(1)
df['temp_34_past'] = df['prev_leftovers_past_3'] - df['prev_leftovers_past_4']
df['temp_23_past'] = df['prev_leftovers_past_2'] - df['prev_leftovers_past_3']
df['temp_12_past'] = df['prev_leftovers_past_1'] - df['prev_leftovers_past_2']
df['temp_01_past'] = df['prev_leftovers'] - df['prev_leftovers_past_1']

df['temp_24_past'] = df['prev_leftovers_past_2'] - df['prev_leftovers_past_4']
df['temp_13_past'] = df['prev_leftovers_past_1'] - df['prev_leftovers_past_3']
df['temp_02_past'] = df['prev_leftovers'] - df['prev_leftovers_past_2']

df['temp_14_past'] = df['prev_leftovers_past_1'] - df['prev_leftovers_past_4']
df['temp_03_past'] = df['prev_leftovers'] - df['prev_leftovers_past_3']
df['temp_04_past'] = df['prev_leftovers'] - df['prev_leftovers_past_4']



# Создаем лаговые прзнаки
for lag in range(12, 21): # начинаем от 12-23 лага что бы на тестовой выборке не тригерилось
    df[f'qt_lag_{lag}'] = df.groupby('nm_id')['qty'].shift(lag)


for lag in range(1, 7):
    df[f'prev0_{lag}'] = df.groupby('nm_id')['prev_leftovers'].shift(lag)


for lag in range(1, 7):
    df[f'temp2_{lag}'] = df.groupby('nm_id')['temp_2'].shift(lag)

for lag in range(1, 7):
    df[f'temp1_{lag}'] = df.groupby('nm_id')['temp_1'].shift(lag)

for lag in range(1, 7):
    df[f'temp_01_past_{lag}'] = df.groupby('nm_id')['temp_01_past'].shift(lag)

for lag in range(1, 7):
    df[f'temp_02_past_{lag}'] = df.groupby('nm_id')['temp_02_past'].shift(lag)

df

,nm_id,dt,qty,price,is_promo,prev_leftovers,prev_leftovers_future_2,prev_leftovers_future_1,temp_2,temp_1,...,temp_01_past_3,temp_01_past_4,temp_01_past_5,temp_01_past_6,temp_02_past_1,temp_02_past_2,temp_02_past_3,temp_02_past_4,temp_02_past_5,temp_02_past_6
0,0045b02f-cd56-4d2f-9d03-c5f40f1da697,2025-03-17,0.0,15555,0,573,573.0,573.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0045b02f-cd56-4d2f-9d03-c5f40f1da697,2025-03-18,0.0,15555,0,573,573.0,573.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0045b02f-cd56-4d2f-9d03-c5f40f1da697,2025-03-19,0.0,15555,0,573,573.0,573.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0045b02f-cd56-4d2f-9d03-c5f40f1da697,2025-03-20,3.0,15555,0,573,576.0,573.0,-3.0,-3.0,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,0045b02f-cd56-4d2f-9d03-c5f40f1da697,2025-03-21,0.0,15555,0,573,576.0,576.0,0.0,0.0,...,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163098,fff52de5-7099-434e-ba06-94be7ca0d629,2025-07-17,NaN,3078,1,15,15.0,15.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
163099,fff52de5-7099-434e-ba06-94be7ca0d629,2025-07-18,NaN,2964,1,15,15.0,15.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
163100,fff52de5-7099-434e-ba06-94be7ca0d629,2025-07-19,NaN,2964,1,15,15.0,15.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
163101,fff52de5-7099-434e-ba06-94be7ca0d629,2025-07-20,NaN,2730,1,15,NaN,15.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# генерируем вторую пачку признаков статистик с исопльзованием окон различной ширины

windows = [3, 5, 7, 14, 30]

for window in windows:
    prefix = f'prev_{window}d_'
    
    # Вычисляем rolling статистики
    roll_stats = df.groupby('nm_id')['prev_leftovers'].rolling(window=window, min_periods=1).agg([
        'mean', 'std', 'min', 'max'
    ]).reset_index(level=0, drop=True)
    
    # Присваиваем ПО ОДНОЙ КОЛОНКЕ
    df[f'{prefix}mean'] = roll_stats['mean']
    df[f'{prefix}std'] = roll_stats['std']
    df[f'{prefix}min'] = roll_stats['min']
    df[f'{prefix}max'] = roll_stats['max']



for window in windows:
    prefix = f'qt_{window}d_'
    
    # Вычисляем rolling статистики
    roll_stats = df.groupby('nm_id')['qt_lag_12'].rolling(window=window, min_periods=1).agg([
        'mean', 'std', 'min', 'max'
    ]).reset_index(level=0, drop=True)
    
    # Присваиваем ПО ОДНОЙ КОЛОНКЕ
    df[f'{prefix}mean'] = roll_stats['mean']
    df[f'{prefix}std'] = roll_stats['std']
    df[f'{prefix}min'] = roll_stats['min']
    df[f'{prefix}max'] = roll_stats['max']


for window in windows:
    prefix = f'temp1_{window}d_'
    
    # Вычисляем rolling статистики
    roll_stats = df.groupby('nm_id')['temp_1'].rolling(window=window, min_periods=1).agg([
        'mean', 'std', 'min', 'max'
    ]).reset_index(level=0, drop=True)
    
    # Присваиваем ПО ОДНОЙ КОЛОНКЕ
    df[f'{prefix}mean'] = roll_stats['mean']
    df[f'{prefix}std'] = roll_stats['std']
    df[f'{prefix}min'] = roll_stats['min']
    df[f'{prefix}max'] = roll_stats['max']


for window in windows:
    prefix = f'temp2_{window}d_'
    
    # Вычисляем rolling статистики
    roll_stats = df.groupby('nm_id')['temp_2'].rolling(window=window, min_periods=1).agg([
        'mean', 'std', 'min', 'max'
    ]).reset_index(level=0, drop=True)
    
    # Присваиваем ПО ОДНОЙ КОЛОНКЕ
    df[f'{prefix}mean'] = roll_stats['mean']
    df[f'{prefix}std'] = roll_stats['std']
    df[f'{prefix}min'] = roll_stats['min']
    df[f'{prefix}max'] = roll_stats['max']

for window in windows:
    prefix = f'temp03_{window}d_'
    
    # Вычисляем rolling статистики
    roll_stats = df.groupby('nm_id')['temp_03_past'].rolling(window=window, min_periods=1).agg([
        'mean', 'std', 'min', 'max'
    ]).reset_index(level=0, drop=True)
    
    # Присваиваем ПО ОДНОЙ КОЛОНКЕ
    df[f'{prefix}mean'] = roll_stats['mean']
    df[f'{prefix}std'] = roll_stats['std']
    df[f'{prefix}min'] = roll_stats['min']
    df[f'{prefix}max'] = roll_stats['max']


for window in windows:
    prefix = f'temp02_{window}d_'
    
    # Вычисляем rolling статистики
    roll_stats = df.groupby('nm_id')['temp_02_past'].rolling(window=window, min_periods=1).agg([
        'mean', 'std', 'min', 'max'
    ]).reset_index(level=0, drop=True)
    
    # Присваиваем ПО ОДНОЙ КОЛОНКЕ
    df[f'{prefix}mean'] = roll_stats['mean']
    df[f'{prefix}std'] = roll_stats['std']
    df[f'{prefix}min'] = roll_stats['min']
    df[f'{prefix}max'] = roll_stats['max']

for window in windows:
    prefix = f'temp01_{window}d_'
    
    # Вычисляем rolling статистики
    roll_stats = df.groupby('nm_id')['temp_01_past'].rolling(window=window, min_periods=1).agg([
        'mean', 'std', 'min', 'max'
    ]).reset_index(level=0, drop=True)
    
    # Присваиваем ПО ОДНОЙ КОЛОНКЕ
    df[f'{prefix}mean'] = roll_stats['mean']
    df[f'{prefix}std'] = roll_stats['std']
    df[f'{prefix}min'] = roll_stats['min']
    df[f'{prefix}max'] = roll_stats['max']
df.shape

(163103, 203)

In [11]:
print(f"Количество уникальных товаров во всей выборке - {df['nm_id'].nunique()}")
print(f"Количество признаков - {df.shape[1] - 2}")

Количество уникальных товаров во всей выборке - 949
Количество признаков - 201


## 3. Разбиение выборки на TRAIN VAL TEST

In [12]:
START_TRAIN_DATE = df_train['dt'].min()
START_VAL_DATE = df_train['dt'].max() - timedelta(days=13)
START_TEST_DATE = df_test['dt'].min()

print(f"Начало тренировочных данных - {START_TRAIN_DATE}")
print(f"Начало вылидационных данных - {START_VAL_DATE}")
print(f"Начало тестовых данных - {START_TEST_DATE}")

Начало тренировочных данных - 2024-07-04 00:00:00
Начало вылидационных данных - 2025-06-24 00:00:00
Начало тестовых данных - 2025-07-08 00:00:00


In [13]:
# Разбиваем выборку на тренировочную, валидационную (14 дней) и тестовую (14 дней)

val = df[(df['dt'] >= START_VAL_DATE) & (df['dt'] < START_TEST_DATE)]
train = df[df['dt'] < START_VAL_DATE]
test = df[df['dt'] >= START_TEST_DATE]

X_train = train.drop(['qty', 'dt'], axis=1)
y_train = train['qty']
X_val = val.drop(['qty', 'dt'], axis=1)
y_val = val['qty']
X_test = test.drop(['qty', 'dt'], axis=1)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(140861, 201)
(9386, 201)
(12856, 201)


In [14]:
test['nm_id'].nunique()

949

## 4. Обучаем модель `Catboost`

In [15]:
# оцениваем ошибку на валидации в процессе обучения с учетом того что нам нужно больше штрафовать те истинные значение где НЕ 0
class WeightedMAE(object):
    """MAE где ошибки на target>0 весят в 7x больше"""
    
    def is_max_optimal(self):
        return False  # Минимизируем (меньше = лучше)
    
    def evaluate(self, approxes, target, weight):
        """Основной расчет метрики"""
        assert len(approxes) == 1
        
        preds = np.array(approxes[0])
        target = np.array(target)
        
        # Абсолютная ошибка
        abs_error = np.abs(target - preds)
        
        # Веса: 7x для target > 0, 1x для target == 0
        weights = np.where(target > 0, 7.0, 1.0)
        
        # Взвешенный MAE
        weighted_mae = np.average(abs_error, weights=weights)
        
        return weighted_mae, np.sum(weights)
    
    def get_final_error(self, error, weight):
        """Финальное значение метрики"""
        return error / weight

In [16]:
# идентификатор товара используем ка категориальный признак
cat_features=['nm_id']

# Инициализируем модель
model = CatBoostRegressor(
    loss_function='MAE',
    eval_metric=WeightedMAE(),
    iterations=2000,
    depth=7,                  
    use_best_model=True,
    cat_features=cat_features)

# Взвешивание примеров: строки с y_train > 0 штрафуем в 7 раз сильнее
# Цель: модель лучше предсказывает НЕ Нулевые продажи!
sample_weights = np.where(y_train > 0, 7.0, 1.0)

train_pool = Pool(X_train, y_train, cat_features=cat_features, weight=sample_weights)
val_pool = Pool(X_val, y_val, cat_features=cat_features)

# запускаем обучение
model.fit(train_pool,
          eval_set=val_pool,
          early_stopping_rounds=50, verbose=10, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.0000175	test: 0.0002405	best: 0.0002405 (0)	total: 438ms	remaining: 14m 35s
10:	learn: 0.0000150	test: 0.0001999	best: 0.0001999 (10)	total: 1.15s	remaining: 3m 27s
20:	learn: 0.0000129	test: 0.0001695	best: 0.0001695 (20)	total: 1.76s	remaining: 2m 46s
30:	learn: 0.0000114	test: 0.0001450	best: 0.0001450 (30)	total: 2.4s	remaining: 2m 32s
40:	learn: 0.0000102	test: 0.0001287	best: 0.0001287 (40)	total: 3.03s	remaining: 2m 24s
50:	learn: 0.0000093	test: 0.0001162	best: 0.0001162 (50)	total: 3.65s	remaining: 2m 19s
60:	learn: 0.0000087	test: 0.0001081	best: 0.0001081 (60)	total: 4.32s	remaining: 2m 17s
70:	learn: 0.0000083	test: 0.0001037	best: 0.0001037 (70)	total: 4.98s	remaining: 2m 15s
80:	learn: 0.0000079	test: 0.0001004	best: 0.0001004 (80)	total: 5.67s	remaining: 2m 14s
90:	learn: 0.0000077	test: 0.0000978	best: 0.0000978 (90)	total: 6.28s	remaining: 2m 11s
100:	learn: 0.0000075	test: 0.0000960	best: 0.0000960 (100)	total: 6.9s	remaining: 2m 9s
110:	learn: 0.0000073	t

## 5. Анализ работы модели и признаков

In [17]:
# Посмотрим важность признаков
important = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
})
important.sort_values('importance', ascending=False).head(50)

,feature,importance
6,temp_2,12.255752
7,temp_1,9.452612
0,nm_id,4.553308
93,qt_14d_mean,3.305740
89,qt_7d_mean,2.835364
134,temp2_14d_std,1.625005
195,temp01_14d_min,1.447973
136,temp2_14d_max,1.417408
124,temp2_3d_max,1.271312
85,qt_5d_mean,1.180946


In [18]:
# Посмотрим строки где модель больше всего ошбается на валидации

# 1. Предсказания на валидации
val_pred = model.predict(X_val)

# 2. Таблица с ошибками
errors_df = pd.DataFrame({
    'nm_id': X_val['nm_id'],  # или другой идентификатор
    'y_true': y_val,
    'y_pred': val_pred,
    'abs_error': np.abs(y_val - val_pred),
})


print("ТОП-10 худших ошибок (abs):")
print(errors_df.nlargest(10, 'abs_error')[['nm_id', 'y_true', 'y_pred', 'abs_error']])


print("\nСтатистика ошибок:")
print(errors_df['abs_error'].describe())


print("\nХудшие nm_id по MAE:")
worst_ids = errors_df.groupby('nm_id')['abs_error'].mean().nlargest(10)
print(worst_ids)


ТОП-10 худших ошибок (abs):
                                       nm_id  y_true     y_pred  abs_error
90894   8a9774f4-c039-4fd1-a77f-fd021f075aea    72.0  20.487244  51.512756
6773    089d52f8-5bb7-4d86-8691-16bc4ed190b2    69.0  23.420116  45.579884
4021    050d003b-8752-4499-9f5b-6c60ba06065f    63.0  30.292732  32.707268
88271   860b7ae6-57f3-4561-92d7-5169fe22126f    48.0  25.840987  22.159013
135519  d278e27c-5805-42a0-bb23-dece06bf0e92    24.0   2.794726  21.205274
105145  a1887a45-6348-4e05-b58e-d726ac626477    45.0  65.157746  20.157746
105150  a1887a45-6348-4e05-b58e-d726ac626477    42.0  61.288426  19.288426
143558  e083c059-a346-4549-96ce-f2d35b47c27a    24.0  43.042836  19.042836
89747   89aee078-a4fc-4765-8cf5-28745ea243e5    33.0  14.821954  18.178046
52981   4d3a1fc5-8230-4338-b243-bb8f4b9ee5db    21.0   3.116632  17.883368

Статистика ошибок:
count    9.386000e+03
mean     4.632566e-01
std      1.744687e+00
min      3.724693e-11
25%      9.433138e-08
50%      3.026211

## 6. Делаем прогноз на тестовой выборке и сохраняем файл `.csv`

In [19]:
# делаем прогноз на тесте и сохраняем ответ
test_pool = Pool(X_test, cat_features=cat_features)

# Предикт и округление
y_test_pred = model.predict(test_pool)

y_test_pred = np.clip(np.round(y_test_pred), 0, None).astype(int)  # целые неотрицательные/

# Результат
test_results = test[['nm_id', 'dt']].copy()
test_results['qty_pred'] = y_test_pred
test_results

sample = df_sample.merge(test_results, on=['nm_id', 'dt'], how='left')
sample.fillna(0, inplace=True)
sample['qty_pred'] = sample['qty_pred'].astype('int')
del sample['qty']
sample = sample.rename(columns={'qty_pred':'qty'})
sample

,nm_id,dt,qty
0,0045b02f-cd56-4d2f-9d03-c5f40f1da697,2025-07-08,0
1,0082ae57-7d24-4d43-b743-3abbd590c5aa,2025-07-08,0
2,00b4b259-c2d5-4122-888a-43a0087fc5f5,2025-07-08,0
3,00d0ffa6-c650-4ddf-9e69-4200ed4e2554,2025-07-08,0
4,00d951b5-ae02-4774-a408-1b98200b1577,2025-07-08,0
...,...,...,...
12851,fe44ffe3-9dcd-4f36-b212-a0f7f35776d9,2025-07-21,0
12852,fef549dd-9fe5-4cf2-b9f0-2539c103a4f2,2025-07-21,0
12853,ff0d747b-bdc1-4628-8163-8fcc27eed4bc,2025-07-21,0
12854,ff518e5b-e28f-41f9-9610-bf9f6bdbe334,2025-07-21,0


In [20]:
# Сохраняем ответ предварительно округлив до значеня кратному трем (поскольку в таргетах преимущественно такие значения)
sample['qty'] = sample['qty'].replace({
    1: 3,
    2: 3,
    4: 3,
    5:6,
    7:6,
    8:9,
    10:9,
    11:12,
    13:12,
    14:15,
    16:15,
    17:18,
    19:18,
    
    })
sample.to_csv('final.csv', index=False)
sample

,nm_id,dt,qty
0,0045b02f-cd56-4d2f-9d03-c5f40f1da697,2025-07-08,0
1,0082ae57-7d24-4d43-b743-3abbd590c5aa,2025-07-08,0
2,00b4b259-c2d5-4122-888a-43a0087fc5f5,2025-07-08,0
3,00d0ffa6-c650-4ddf-9e69-4200ed4e2554,2025-07-08,0
4,00d951b5-ae02-4774-a408-1b98200b1577,2025-07-08,0
...,...,...,...
12851,fe44ffe3-9dcd-4f36-b212-a0f7f35776d9,2025-07-21,0
12852,fef549dd-9fe5-4cf2-b9f0-2539c103a4f2,2025-07-21,0
12853,ff0d747b-bdc1-4628-8163-8fcc27eed4bc,2025-07-21,0
12854,ff518e5b-e28f-41f9-9610-bf9f6bdbe334,2025-07-21,0


## 7. Визуализация предсказаний

In [21]:
def pr(nm_id):
    item_data = df[df['nm_id'] == nm_id].sort_values('dt')
    test_data = sample[sample['nm_id'] == nm_id]
    train_max_date = df['dt'].max() - pd.Timedelta(days=14)
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Train бары
    fig.add_trace(go.Bar(x=item_data['dt'], y=item_data['qty'], 
                        name='Продажи train', marker_color='green', opacity=0.9),
                 secondary_y=False)
    
    # Test бары
    if not test_data.empty:
        fig.add_trace(go.Bar(x=test_data['dt'], y=test_data['qty'], 
                           name='Продажи test', marker_color='blue', opacity=0.9),
                     secondary_y=False)
    
    # ЛИНИЯ РАЗДЕЛИТЕЛЯ (add_shape вместо add_vline)
    fig.add_shape(type="line",
                  x0=train_max_date, x1=train_max_date,
                  y0=0, y1=1, yref="paper",
                  line=dict(color="red", width=3, dash="dash"))

    
    # Остатки
    fig.add_trace(go.Bar(x=item_data['dt'], y=item_data['prev_leftovers'].fillna(0),
                        name='Остатки', marker_color='red', opacity=0.5),
                 secondary_y=True)
    
    # Настройка
    fig.update_layout(
        title=f'Продажи и остатки товара {nm_id}',
        hovermode='x unified',
        legend=dict(x=0.01, y=0.99),
        height=500,
        showlegend=True
    )
    fig.update_xaxes(title_text="Дата")
    fig.update_yaxes(title_text="Продажи (qty)", secondary_y=False)
    fig.update_yaxes(title_text="Остатки", secondary_y=True)
    
    fig.show()


In [22]:
nm_id = df_test['nm_id'].unique()[3]
pr(nm_id)

## 8. Bonus Part (что бы поднять ручками score еще немножечко)

In [23]:
# слишком высокая сумма продаж модель прогнозирует за тестовый период по сравнению с тренировочным периодом
nm_id = '154d1322-bd30-496c-9374-6dbd35550669'
pr(nm_id)

In [24]:
# Подправим выбросы в тесте
sample.loc[sample['nm_id'] == '5dcc404b-aea3-42d9-8b1c-a2379e5e19c7', 'qty'] = 0
sample.loc[sample['nm_id'] == '154d1322-bd30-496c-9374-6dbd35550669', 'qty'] = 0
sample.to_csv('final.csv', index=False)

In [25]:
nm_id = '154d1322-bd30-496c-9374-6dbd35550669'
print(nm_id)
pr(nm_id)

154d1322-bd30-496c-9374-6dbd35550669
